While going through our script we will gradually understand the use of this packages

In [1]:
import tensorflow as tf #no need to describe ;)
import numpy as np #allows array operation
import pandas as pd #we will use it to read and manipulate files and columns content

from nltk.corpus import stopwords #provides list of english stopwords
stop = stopwords.words('english')

C:\Users\Admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


To do this notebook we will use New York Times user comments (from Kaggle Datasets).
When we will create the language classifier we will use other data but now let's rely on an english natural language source, so now we read the data.


In [2]:
train = pd.read_csv('CommentsApril2017.csv')


C:\Users\Admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Let's have a quick look at the data trying to find what is the column that we need.
Looks like commentBody is the right candidate.

In [3]:
train.head() 

approveDate                                        commentBody   commentID  \
0   1491245186  This project makes me happy to be a 30+ year T...  22022598.0   
1   1491188619  Stunning photos and reportage. Infuriating tha...  22017350.0   
2   1491188617  Brilliant work from conception to execution. I...  22017334.0   
3   1491167820  NYT reporters should provide a contributor's l...  22015913.0   
4   1491167815     Could only have been done in print. Stunning.   22015466.0   

   commentSequence commentTitle commentType    createDate  depth  \
0         22022598        <br/>     comment  1.491237e+09      1   
1         22017350          NaN     comment  1.491180e+09      1   
2         22017334        <br/>     comment  1.491179e+09      1   
3         22015913        <br/>     comment  1.491150e+09      1   
4         22015466        <br/>     comment  1.491147e+09      1   

   editorsSelection  parentID       ...           userLocation userTitle  \
0             False       0.0       ...          Riverside, CA       NaN   
1             False       0.0       ...                  <br/>       NaN   
2             False       0.0       ...             Raleigh NC       NaN   
3             False       0.0       ...          Missouri, USA       NaN   
4             False       0.0       ...        Tucson, Arizona       NaN   

  userURL  inReplyTo                 articleID  sectionName  newDesk  \
0     NaN          0  58def1347c459f24986d7c80      Unknown  Insider   
1     NaN          0  58def1347c459f24986d7c80      Unknown  Insider   
2     NaN          0  58def1347c459f24986d7c80      Unknown  Insider   
3     NaN          0  58def1347c459f24986d7c80      Unknown  Insider   
4     NaN          0  58def1347c459f24986d7c80      Unknown  Insider   

   articleWordCount printPage  typeOfMaterial  
0             716.0         2            News  
1             716.0         2            News  
2             716.0         2            News  
3             716.0         2            News  
4             716.0         2            News  

[5 rows x 34 columns]

now we first put everything to lowercase and then replace undesired characters

In [4]:
train['commentBody_lower'] = train["commentBody"].str.lower()
train['commentBody_no_punctiation'] = train['commentBody_lower'].str.replace('[^\w\s]','')

let's check how the text looks like now!
Well everything is lowercase and no "ugly characters"

In [5]:
train['commentBody_no_punctiation'].head() 

0    this project makes me happy to be a 30 year ti...
1    stunning photos and reportage infuriating that...
2    brilliant work from conception to execution iv...
3    nyt reporters should provide a contributors li...
4         could only have been done in print stunning 
Name: commentBody_no_punctiation, dtype: object

Now we remove stopwords and then fill empy cells with "fillna" word.

In [6]:
train['commentBody_no_stopwords'] = train['commentBody_no_punctiation'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [7]:
train["commentBody_no_stopwords"] = train["commentBody_no_stopwords"].fillna("fillna")

This is how our cleaned text looks like we can see that everything is lowercase and the stopwords are missing, for example "this". Now let's go back to slides.


In [8]:
train['commentBody_no_stopwords'].head()

0    project makes happy 30 year times subscriber c...
1    stunning photos reportage infuriating trump ad...
2    brilliant work conception execution ive never ...
3    nyt reporters provide contributors link crowdf...
4                            could done print stunning
Name: commentBody_no_stopwords, dtype: object

In [65]:
tf_train = train


We first assign our current data frame to another to keep track of our work then we read the first sentence and count words that result to be 21

In [66]:
tf_train['commentBody_no_stopwords'][1]

'stunning photos reportage infuriating trump admistrations draconian reinstatement global gag order prevent men women receiving appropriate family planning advice obviously desperately needed'

In [67]:
tf_train['commentBody_no_stopwords'][1].count(' ')

21

In [68]:
max_features=5000 #we set maximum number of words to 5000
maxlen=100 #and maximum sequence length to 100


In [69]:
tok = tf.keras.preprocessing.text.Tokenizer(num_words=max_features) #tokenizer step

In [70]:
tok.fit_on_texts(list(tf_train['commentBody_no_stopwords'])) #fit to cleaned text

In [71]:
tf_train=tok.texts_to_sequences(list(tf_train['commentBody_no_stopwords'])) #this is how we create sequences

In [72]:
print(type(tf_train)) #we see that the type is now list
print(len(tf_train[1])) #we see that the number of words of the sentence is decreased to 16
tf_train[1] #and this is how our sentece looks like now, exactly a sequence of integers

<class 'list'>
16


[4916,
 3398,
 1,
 528,
 330,
 1253,
 201,
 88,
 3266,
 1682,
 138,
 1827,
 1362,
 854,
 2961,
 406]

In [73]:
tf_train=tf.keras.preprocessing.sequence.pad_sequences(tf_train, maxlen=maxlen) #let's execute pad step 

In [74]:
print(len(tf_train[1]))
tf_train[1] #this is how our sentece looks like after the pad step we don't have anymore 16 words but 100 (equivalent to maxlen)

100


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0, 4916, 3398,    1,  528,
        330, 1253,  201,   88, 3266, 1682,  138, 1827, 1362,  854, 2961,
        406])

In [75]:
train['commentBody_no_stopwords'][1] #let's look at the input text

'stunning photos reportage infuriating trump admistrations draconian reinstatement global gag order prevent men women receiving appropriate family planning advice obviously desperately needed'

In [76]:
tf_train = pd.DataFrame(tf_train)



In [77]:
tf_train.head() #let's look at the final matrix that will use as an input for our deep learning algorithms, do you remember 
#how original text looked like?

0   1   2   3   4   5   6   7   8   9   ...     90    91    92    93    94  \
0   0   0   0   0   0   0   0   0   0   0  ...      0  1797   181   586   915   
1   0   0   0   0   0   0   0   0   0   0  ...    201    88  3266  1682   138   
2   0   0   0   0   0   0   0   0   0   0  ...   1788   402   146   565  2122   
3   0   0   0   0   0   0   0   0   0   0  ...   3766    78  2705  2192    67   
4   0   0   0   0   0   0   0   0   0   0  ...      0     0     0     0     0   

     95    96    97    98    99  
0   148    66   255   656   273  
1  1827  1362   854  2961   406  
2  1240  2790  1289  2034  1118  
3   442  3267   813   924  2791  
4     0    15   134  2581  4916  

[5 rows x 100 columns]